In [1]:
import tensorflow as tf
import numpy as np

class SelfAttenModel(object):
    
    def __init__(self,
                 batch_size=40, 
                 vocab_size=200,
                 layer_num=2, 
                 embedding_size=100, 
                 keep_prob=0.8, 
                 max_sequence_length=10,
                 num_units=128,
                 d_a=350,
                 r=30):
        
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.layer_num = layer_num
        self.embedding_size = embedding_size
        self.keep_prob = keep_prob
        self.n = self.max_sequence_length = max_sequence_length
        self.u = self.num_units = num_units
        self.d_a = d_a
        self.r = r
        
        self._build_placeholder()
        self._build_model()
            
    def _build_placeholder(self):
        self.sources = tf.placeholder(name='sources', shape=[self.batch_size, self.max_sequence_length], dtype=tf.int64)
        self.labels = tf.placeholder(name='labels', shape=[self.batch_size], dtype=tf.int64)
        

    def _build_single_cell(self):
        cell = tf.contrib.rnn.BasicLSTMCell(self.num_units)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=self.keep_prob)
        return cell
    
    def _build_model(self):
        # Word embedding #
        with tf.variable_scope("embedding"):
            initializer = tf.contrib.layers.xavier_initializer()
            embeddings = tf.get_variable(name="embedding_encoder",
                                                shape=[self.vocab_size, self.embedding_size], 
                                                dtype=tf.float32,
                                                initializer=initializer,
                                                trainable=True)

            input_embeddings = tf.nn.embedding_lookup(params=embeddings,
                                                      ids=self.sources)

        # Bidirectional rnn #
        with tf.variable_scope("bidirectional_rnn"):
            cell_forward = self._build_single_cell()
            cell_backward = self._build_single_cell()
            
            # outputs is state 'H'
            outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell_forward, 
                                                              cell_bw=cell_backward, 
                                                              inputs=input_embeddings,
                                                              dtype=tf.float32)
            
            outputs = tf.concat(outputs, -1)
            
        # Summation weights #
        with tf.variable_scope("summation_weights"):
            initializer = tf.contrib.layers.xavier_initializer()
            W_s1 = tf.get_variable(name="W_s1", shape=[2*self.u, self.d_a], initializer=initializer)
            W_s2 = tf.get_variable(name='W_s2', shape=[self.d_a, self.r],initializer=initializer)
            
            outputs_concat = tf.reshape(outputs, shape=[-1, 2*self.u])
            outputs_mid = tf.tanh(tf.matmul(outputs_concat, W_s1))
            outputs_fin = tf.matmul(outputs_mid, W_s2)
            outputs_fin = tf.reshape(outputs_fin, [-1, self.n, self.r])
            outputs_fin = tf.transpose(outputs_fin, perm=[0,2,1])
            A = tf.nn.softmax(outputs_fin)
            print(A)
        
            
            
def main():
    model = SelfAttenModel()

if __name__ == '__main__':
    main()
    

Tensor("bidirectional_rnn/concat:0", shape=(40, 10, 256), dtype=float32)
Tensor("summation_weights/Reshape:0", shape=(400, 256), dtype=float32)
Tensor("summation_weights/Reshape_3:0", shape=(40, 30, 10), dtype=float32)
